In [1]:
import h5py
import sys
import os
import glob
import csv
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from hdfs import InsecureClient

Go through the base path and find all the .h5 files that are present there then save them to a csv file in order to load them later into spark

TODO: this actually needs to be reworked to create a table based on the hdfs file directory instead of local, however it doesn't matter as long as we don't change the files just run the code and pretend it does the right thing for now

In [2]:
basedir = '../MillionSongSubset/data'
os.listdir(basedir)

['A', 'B', 'files.csv']

In [3]:
ext='.h5'
all_files = []
for root, dirs, files in os.walk(basedir):
    files = glob.glob(os.path.join(root,'*'+ext))
    all_files.append(files)

flat_list = [item for sublist in all_files for item in sublist]

In [4]:
file = '/files.csv'
with open(basedir + file, 'w', newline='') as myfile:
    for line in flat_list:
        myfile.write(line)
        myfile.write('\n')

Set up the spark application

In [5]:
# calculate the number of partitions we want for the rdds
num_nodes = 3
num_rep = 2 * num_nodes

conf = (SparkConf()
   .setMaster("spark://192.168.2.110:7077")
   .setAppName("Group14")
   .set("spark.executor.cores", 2)
   .set("spark.pytspark.python","python3"))

sc = SparkContext(conf = conf)

In [6]:
sqlContext = SQLContext(sc)

code to convert filenames to actual file content

In [7]:
# good methods to explore the h5 inread: .keys(), .attrs.items() see documentation of h5py
import tables




def get_title(file, idx=0):
    return file['metadata']['songs']['title'][idx].decode("utf-8")

def get_artist_name(file, idx=0):
    return file['metadata']['songs']['artist_name'][idx].decode("utf-8")


def get_artist_familiarity(h5,songidx=0):
    """
    Get artist familiarity from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_familiarity'][songidx].decode("utf-8")

def get_artist_hotttnesss(h5,songidx=0):
    """
    Get artist hotttnesss from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_hotttnesss'][songidx].decode("utf-8")


def get_artist_id(h5,songidx=0):
    """
    Get artist id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_id'][songidx].decode("utf-8")

def get_artist_mbid(h5,songidx=0):
    """
    Get artist musibrainz id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_mbid'][songidx].decode("utf-8")

def get_artist_playmeid(h5,songidx=0):
    """
    Get artist playme id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_playmeid'][songidx].decode("utf-8")

def get_artist_7digitalid(h5,songidx=0):
    """
    Get artist 7digital id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_7digitalid'][songidx].decode("utf-8")

def get_artist_latitude(h5,songidx=0):
    """
    Get artist latitude from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_latitude'][songidx].decode("utf-8")

def get_artist_longitude(h5,songidx=0):
    """
    Get artist longitude from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_longitude'][songidx].decode("utf-8")

def get_artist_location(h5,songidx=0):
    """
    Get artist location from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_location'][songidx].decode("utf-8")


def get_release(h5,songidx=0):
    """
    Get release from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['release'][songidx].decode("utf-8")

def get_release_7digitalid(h5,songidx=0):
    """
    Get release 7digital id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['release_7ditgitalid'][songidx].decode("utf-8")

def get_song_id(h5,songidx=0):
    """
    Get song id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['song_id'][songidx].decode("utf-8")

def get_song_hotttnesss(h5,songidx=0):
    """
    Get song hotttnesss from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['song_hotnesss'][songidx].decode("utf-8")


def get_track_7digitalid(h5,songidx=0):
    """
    Get track 7digital id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['track_7ditgitalid'][songidx].decode("utf-8")

def get_similar_artists(h5,songidx=0):
    """
    Get similar artists array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.similar_artists[h5.root.metadata.songs.cols.idx_similar_artists[songidx]:]
    return h5.root.metadata.similar_artists[h5.root.metadata.songs.cols.idx_similar_artists[songidx]:
                                            h5.root.metadata.songs.cols.idx_similar_artists[songidx+1]]

def get_artist_terms(h5,songidx=0):
    """
    Get artist terms array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.artist_terms[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:]
    return h5.root.metadata.artist_terms[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:
                                            h5.root.metadata.songs.cols.idx_artist_terms[songidx+1]]

def get_artist_terms_freq(h5,songidx=0):
    """
    Get artist terms array frequencies. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.artist_terms_freq[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:]
    return h5.root.metadata.artist_terms_freq[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:
                                              h5.root.metadata.songs.cols.idx_artist_terms[songidx+1]]

def get_artist_terms_weight(h5,songidx=0):
    """
    Get artist terms array frequencies. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.artist_terms_weight[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:]
    return h5.root.metadata.artist_terms_weight[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:
                                                h5.root.metadata.songs.cols.idx_artist_terms[songidx+1]]

def get_analysis_sample_rate(h5,songidx=0):
    """
    Get analysis sample rate from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['analysis_sample_rate'][songidx].decode("utf-8")


def get_audio_md5(h5,songidx=0):
    """
    Get audio MD5 from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['audio_md5'][songidx].decode("utf-8")


def get_danceability(h5,songidx=0):
    """
    Get danceability from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['danceability'][songidx].decode("utf-8")


def get_duration(h5,songidx=0):
    """
    Get duration from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['duration'][songidx].decode("utf-8")


def get_end_of_fade_in(h5,songidx=0):
    """
    Get end of fade in from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['end_of_fade_in'][songidx].decode("utf-8")


def get_energy(h5,songidx=0):
    """
    Get energy from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['energy'][songidx].decode("utf-8")


def get_key(h5,songidx=0):
    """
    Get key from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['key'][songidx].decode("utf-8")


def get_key_confidence(h5,songidx=0):
    """
    Get key confidence from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['key_confidence'][songidx].decode("utf-8")


def get_loudness(h5,songidx=0):
    """
    Get loudness from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['loudness'][songidx].decode("utf-8")


def get_mode(h5,songidx=0):
    """
    Get mode from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['mode'][songidx].decode("utf-8")


def get_mode_confidence(h5,songidx=0):
    """
    Get mode confidence from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['mode_confidence'][songidx].decode("utf-8")


def get_start_of_fade_out(h5,songidx=0):
    """
    Get start of fade out from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['start_of_fade_out'][songidx].decode("utf-8")


def get_tempo(h5,songidx=0):
    """
    Get tempo from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['tempo'][songidx].decode("utf-8")

def get_time_signature(h5,songidx=0):
    """
    Get signature from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['time_signature'][songidx].decode("utf-8")

def get_time_signature_confidence(h5,songidx=0):
    """
    Get signature confidence from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['time_signature_confidence'][songidx].decode("utf-8")

def get_track_id(h5,songidx=0):
    """
    Get track id from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['track_id'][songidx].decode("utf-8")

def get_year(h5,songidx=0):
    """
    Get release year from a HDF5 song file, by default the first song in it
    """
    return h5['musicbrainz']['songs']['year'][songidx].decode("utf-8")



# todo: implement functions to get other attributes here!!

In [8]:
# import hdf5_getters
import tables
import requests
import io

# right now only mapping for the title needs to be changed to extract all the other features
def f(x):
    # set up the url where the data is stored (yes it's ugly)
    base_hdfs = '/LDSA/data'
    url = 'http://192.168.2.110:50070/webhdfs/v1'
    op = '?op=OPEN'
    full_path = url + base_hdfs + x[2:] + op
    # read in song file from hdfs api and then extract the wanted attributes
    with h5py.File(io.BytesIO(requests.get(url = full_path).content), 'r') as f:
        song = []
        song.append(get_title(f))
        song.append(get_artist_name(f))
    return song

In [9]:
# todo: set the number of partitions here corectly as calculated above in order to maintain speed
file_paths = sc.textFile(basedir + file).map(lambda x: f(x))

In [10]:
file_paths.collect()

[['The Matter With You (Part I_ Ii_ Iii)', 'Avril'],
 ['Requiem Mass_ K. 626_ 1791: Agnus Dei (Larghetto) (Mozart)',
  'Choir & Great Symphony Orchestra of the All-Union Radio_ N. Shpiller_ S. Krassovsky_ V. Gagarina & N'],
 ["Diamonds Are A Girl's Best Friend", 'Gloria De Haven'],
 ["Don't Owe Me Nothin'", 'Sticky Fingaz & Omar Epps'],
 ['Lose Your Money', 'Delbert McClinton'],
 ['Gotta Let It Go', 'Elva Hsiao'],
 ['Paint It Black', 'Chris Farlowe'],
 ['Fearless [Interlude] (Explicit Album Version)', 'Bone Thugs-N-Harmony'],
 ['Skyswimmer', 'Enter the Haggis'],
 ['He Has Risen (Communication Album Version)', 'DeGarmo & Key'],
 ['Moon Loop (Improvisation)', 'Porcupine Tree'],
 ['Old Hobo', 'Furry Lewis'],
 ['Tel Que Tu Es', 'Charlotte Gainsbourg'],
 ['Heidi', 'Frank Chacksfield'],
 ['Non Conformist Acclaimed', 'Agnes'],
 ["I Just Need Your Lovin' - Original", 'Chris Farlowe'],
 ['Space Oddity (1997 Digital Remaster)', 'David Bowie'],
 ['Here Comes Your Iceman (LP Version)', 'Joe Turner

convert the rdd (api version 1) to a dataframe (api version 2)

In [ ]:
# todo: add the needed attributes here that we need..
df = sqlContext.createDataFrame(file_paths, schema=['title', 'artist_name'])

In [ ]:
df.printSchema()

start with the data processing

In [ ]:
df.show()

In [ ]:
sc.stop()